[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/seoul-bike-analysis/blob/master/bike_station_scrape.ipynb)

# 위경도 데이터 수집하기
* 따릉이 대여소 정보 가져오기
* 수집하고자 하는 데이터 위치 : https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo=1
* colab 경로 : https://colab.research.google.com/github/corazzon/seoul-bike-analysis/blob/master/crawling.ipynb

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# random은 랜덤한 시간차를 두고 가져오기 위해 사용한다.
import random
# time 으로 시간을 구한다.
import time
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd
import numpy as np

In [2]:
# 사이트 주소
base_url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

### html 페이지 받아오기

In [3]:
pnum = 1
response = requests.get( f"{base_url}{pnum}")
html = bs(response.text)

In [4]:
def requests_url(pnum):
    response = requests.get( f"{base_url}{pnum}")
    html = bs(response.text)
    trs = html.select("table.psboard1 > tbody > tr")
    
    if trs:
        return trs
    else:
        return False

In [5]:
requests_url(-1)

False

In [6]:
trs = requests_url(1)
# trs

In [7]:
trs[0]

<tr>
<td class="pl10"><a href="#" param-data="37.54179382,127.12474823">1001. 광진교 남단 사거리(디지털프라자앞)</a></td>
<td class="pl10">
                        운영중</td>
<td class="tr">15</td>
<td class="tr">0</td>
<td class="tbleft mhid"><span>서울특별시 강동구 올림픽로 674 로석빌딩 </span></td>
</tr>

### 수집하고자 하는 데이터 찾기

In [8]:
def parse_tr(tr):
    try:
        info = []
        tds = tr.select("td")
        for td in tds:
            td_text = td.get_text(strip=True)
            info.append(td_text)
        
        longlat = tds[0].select("a")[0]["param-data"]
        info.append(longlat)
        return info
    except:
        return False
    
parse_tr(trs[1])

['1002. 해공공원(천호동)',
 '운영중',
 '10',
 '0',
 '서울특별시 강동구 올림픽로 702 해공도서관',
 '37.54526138,127.12594604']

In [9]:
def parse_table(trs):
    stations = []
    for tr in trs:
        info = parse_tr(tr)
        if info :
            stations.append(info)
    return stations

parse_table(trs)

[['1001. 광진교 남단 사거리(디지털프라자앞)',
  '운영중',
  '15',
  '0',
  '서울특별시 강동구 올림픽로 674 로석빌딩',
  '37.54179382,127.12474823'],
 ['1002. 해공공원(천호동)',
  '운영중',
  '10',
  '0',
  '서울특별시 강동구 올림픽로 702 해공도서관',
  '37.54526138,127.12594604'],
 ['1003. 해공도서관앞',
  '운영중',
  '20',
  '0',
  '서울특별시 강동구 올림픽로 702 해공도서관',
  '37.54395676,127.12548828'],
 ['1004. 삼성광나루아파트 버스정류장',
  '운영중',
  '10',
  '4',
  '서울특별시 강동구 올림픽로 812',
  '37.55332947,127.12886810'],
 ['1006. 롯데캐슬 115동앞',
  '운영중',
  '15',
  '0',
  '서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트',
  '37.55486679,127.14279938']]

In [10]:
def get_page(pnum):
    trs = requests_url(pnum)
    try:
        stations = parse_table(trs)
        return stations
    except:
        return False

In [11]:
get_page(409)

False

In [12]:
from tqdm import tqdm, trange
start_page = 1
end_page = 410

station_list = []

for page_no in trange(start_page, end_page+1):
    # 0~1초 시간간격을 두고 가져옵니다.
    sleep_time = np.random.uniform(0, 1)
    time.sleep(sleep_time)
    
    stations = get_page(pnum)
    if stations:
        station_list.extend(stations)

100%|██████████| 410/410 [08:11<00:00,  1.20s/it]


In [13]:
pd.DataFrame(station_list).head()

,0,1,2,3,4,5
0,1001. 광진교 남단 사거리(디지털프라자앞),운영중,15,0,서울특별시 강동구 올림픽로 674 로석빌딩,"37.54179382,127.12474823"
1,1002. 해공공원(천호동),운영중,10,0,서울특별시 강동구 올림픽로 702 해공도서관,"37.54526138,127.12594604"
2,1003. 해공도서관앞,운영중,20,0,서울특별시 강동구 올림픽로 702 해공도서관,"37.54395676,127.12548828"
3,1004. 삼성광나루아파트 버스정류장,운영중,10,4,서울특별시 강동구 올림픽로 812,"37.55332947,127.12886810"
4,1006. 롯데캐슬 115동앞,운영중,15,0,서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트,"37.55486679,127.14279938"


In [14]:
header = ['대여소', '운영여부', '거치대수', '대여가능', '주소', '위경도']
df = pd.DataFrame(station_list, columns = header)
df.shape

(2050, 6)

In [15]:
df.head()

,대여소,운영여부,거치대수,대여가능,주소,위경도
0,1001. 광진교 남단 사거리(디지털프라자앞),운영중,15,0,서울특별시 강동구 올림픽로 674 로석빌딩,"37.54179382,127.12474823"
1,1002. 해공공원(천호동),운영중,10,0,서울특별시 강동구 올림픽로 702 해공도서관,"37.54526138,127.12594604"
2,1003. 해공도서관앞,운영중,20,0,서울특별시 강동구 올림픽로 702 해공도서관,"37.54395676,127.12548828"
3,1004. 삼성광나루아파트 버스정류장,운영중,10,4,서울특별시 강동구 올림픽로 812,"37.55332947,127.12886810"
4,1006. 롯데캐슬 115동앞,운영중,15,0,서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트,"37.55486679,127.14279938"


In [16]:
df.tail()

,대여소,운영여부,거치대수,대여가능,주소,위경도
2045,1001. 광진교 남단 사거리(디지털프라자앞),운영중,15,0,서울특별시 강동구 올림픽로 674 로석빌딩,"37.54179382,127.12474823"
2046,1002. 해공공원(천호동),운영중,10,0,서울특별시 강동구 올림픽로 702 해공도서관,"37.54526138,127.12594604"
2047,1003. 해공도서관앞,운영중,20,0,서울특별시 강동구 올림픽로 702 해공도서관,"37.54395676,127.12548828"
2048,1004. 삼성광나루아파트 버스정류장,운영중,10,5,서울특별시 강동구 올림픽로 812,"37.55332947,127.12886810"
2049,1006. 롯데캐슬 115동앞,운영중,15,0,서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트,"37.55486679,127.14279938"


In [17]:
df["대여소번호"] = df["대여소"].str.split(".", expand=True)[0]
df["대여소명"] = df["대여소"].str.split(".", expand=True)[1]

In [18]:
df["위도"] = df["위경도"].str.split(",", expand=True)[0].astype(float)
df["경도"] = df["위경도"].str.split(",", expand=True)[1].astype(float)

In [19]:
df.head(1)

,대여소,운영여부,거치대수,대여가능,주소,위경도,대여소번호,대여소명,위도,경도
0,1001. 광진교 남단 사거리(디지털프라자앞),운영중,15,0,서울특별시 강동구 올림픽로 674 로석빌딩,"37.54179382,127.12474823",1001,광진교 남단 사거리(디지털프라자앞),37.541794,127.124748


In [20]:
# df = df.drop(["대여소", "위경도"], axis=1)

In [21]:
df.columns

Index(['대여소', '운영여부', '거치대수', '대여가능', '주소', '위경도', '대여소번호', '대여소명', '위도',
       '경도'],
      dtype='object')

In [22]:
df = df[['대여소번호', '대여소명', '운영여부', '거치대수', '대여가능', '주소', '위도', '경도']]

In [23]:
df.to_csv('bike_rent_station.csv', index=False)

In [24]:
# 파일이 제대로 생성되었는지 확인
pd.read_csv('bike_rent_station.csv').head()

,대여소번호,대여소명,운영여부,거치대수,대여가능,주소,위도,경도
0,1001,광진교 남단 사거리(디지털프라자앞),운영중,15,0,서울특별시 강동구 올림픽로 674 로석빌딩,37.541794,127.124748
1,1002,해공공원(천호동),운영중,10,0,서울특별시 강동구 올림픽로 702 해공도서관,37.545261,127.125946
2,1003,해공도서관앞,운영중,20,0,서울특별시 강동구 올림픽로 702 해공도서관,37.543957,127.125488
3,1004,삼성광나루아파트 버스정류장,운영중,10,4,서울특별시 강동구 올림픽로 812,37.553329,127.128868
4,1006,롯데캐슬 115동앞,운영중,15,0,서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트,37.554867,127.142799


In [25]:
pd.read_csv('bike_rent_station.csv').shape

(2050, 8)